# Tutorial Basics Noten Part 2: Statistik

Inhaltsverzeichnis: 
1. Abfrage von Metadaten und einfachen statischen Angaben
2. Häufigkeiten: Tonhöhen, Tonhöhenklassen, Tondauern, metrische Akzente
3. Zweidimensionale Häufigkeitsverteilungen
4. Intervalle im einstimmigen Verlauf 
5. Aufgaben

In diesem Tutorial werden Sie anhand von Musikbeispielen einfache computergestützten Möglichkeiten statistischer Abfragen kennenlernen. 

Das Durcharbeiten und Nachvollziehen des Tutoriums soll es Ihnen ermöglichen, mit den vorgestellten computergestützten Methoden eigene Musikbeispiele (Notendateien) zu untersuchen und Vergleiche zwischen verschiedenen Musikstücken durchzuführen. 
Dabei geht es immer auch um die Formulierung bestimmter analytischer Fragestellungen und die Interpretation der jeweiligen Ergebnisse.

In [ ]:
import sys
import os
sys.path.append(os.getcwd().replace(os.path.join('music_xml_parser', 'ipynb'), ''))

import music_xml_parser as mp
from music21 import * 
import csv
from IPython.display import HTML, display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Mit diesen Befehlen wird nicht nur 'music21', 
# sondern darüber hinaus die Bibliotheken 'numpy' und 'pandas' für statische Auswertungen 
# sowie 'matplotlib' für grafische Darstellungen geladen. 

# mit den folgenden Befehlen wird die Formtierung für die Tabellen festgelegt, 
# die weiter unten dargestellt werden: 

pd.set_option('display.max_rows', 9999)
pd.set_option('display.max_columns', 9999)
pd.set_option('display.width', 9999)

environment.set('autoDownload', 'allow')

# Und zuletzt Befehle für den Datenexport: 




In [ ]:
%matplotlib
xml_file = 'PrJode_Jos1102_COM_1-5_MissaLasol_002_00137.xml'
# xml_file = 'BeLuva_Op59_1-3_1-4_StringQuar_003_00129.xml'
m_df, measure_onset_df = mp.parse.with_xml_file(file_name=xml_file,
                                                plot_pianoroll=True,
                                                plot_inline_ipynb=False,
                                                save_at=None,
                                                save_file_name=None,
                                                do_save=False, 
                                                x_axis_res=2, 
                                                get_measure_onset=True)




In [ ]:
%matplotlib
##Testing filter
xml_file = 'PrJode_Jos1102_COM_1-5_MissaLasol_002_00137.xml'

filter_dict_t = { 'Measure':'2-5', 'PartID':'1-4'}


m_df_2, measure_onset_df_2 = mp.parse.with_xml_file(file_name=xml_file,
                                                    plot_pianoroll=True,
                                                    plot_inline_ipynb=False,
                                                    save_at=None,
                                                    save_file_name=None,
                                                    do_save=False, 
                                                    x_axis_res=2, 
                                                    get_measure_onset=True,filter_dict=filter_dict_t)




m_df_2

In [ ]:
m_df

Man kann sich das Stück auch anhören - allerdings nur in einer Version mit MIDI-Klängen (und nur in den Browsern Chrome und Firefox): 

### 1. Abfrage von Metadaten und einfachen statischen Angaben

Alle in der MusicXML gespeicherte Metadaten zum untersuchten Werk (Komponist, Werktitel, Entstehungsjahr, usw.) können mit dem folgenden Befehl angezeigt werden:

Nun die einfachen statistischen Abfragen aus dem letzten Tutorial: Anzahl der Stimmen, Anzahl der Töne (insgesamt und pro Stimmen), Länge in Takten, Ambitus, die wir bereits im Grundlagen-Tutorial kennengelernt haben:

In [ ]:
v = m_df[['PartID','Part Name']].drop_duplicates().to_numpy()
mp.utils.display_table(data=v, 
                       columns=['Part ID', 'Part Names'])

# Anzahl der Stimmen
# 'local' wird als Name (Variable) für die geladene ('geparste') Notendatei gewählt

In [ ]:
m = m_df['Measure'].to_numpy(dtype=int)
max(m)
# Länge in Takten

In [ ]:
n_notes, c_notes = np.unique(m_df['PartID'], return_counts=True)
data = [[i, c] for i, c in zip(n_notes, c_notes) ]
mp.utils.display_table(data=data, 
                       columns=['Part ID', '# Notes'])
# parser takes tied notes in consideration 
# Anzahl der Töne per Stimme

In [ ]:
ambitus = mp.analyse.ambitus(m_df)
mp.utils.display_table(data=ambitus, 
                       columns=['Part ID', 'min', 'max', 'Semitones Difference'])

# Ambitus per Stimme

### 2. Häufigkeiten: Tonhöhen, Tonhöhenklassen, Tondauern

Zur Charakterisierung einzelner Musikstücke und zum Vergleich zwischen verschiedenen Musikstücken kann es sinnvoll sein, die Häufigkeit bestimmter Elemente (Tonhöhen, Dauernwerte, Akkorde etc.) zu bestimmen. Für solche Fragen lassen Häufigkeitstabellen und grafische Darstellungen, sog. Histogramme erstellen. 

#### 2.1 Tonhöhen

Welche Töne tauchen wie häufig auf? Wie diatonisch ist der Tonraum der Missa, wie viele zusätzliche chromatische Töne tauchen auf?

In [ ]:
%matplotlib inline

pitch_hist = mp.analyse.pitch_histogram(m_df, do_plot=True, visulize_midi_range=None)
mp.utils.display_table(data=pitch_hist, 
                       columns=['MIDI', 'Pitch','Occurrences'])
# ToDo: pitch cols


In [ ]:
ph = mp.analyse.pitch_histogram(m_df, do_plot=True, visulize_midi_range=[50, 90])

In [ ]:

ph2 = mp.analyse.pitch_histogram(m_df, do_plot=True, do_plot_full_axis=False, visulize_midi_range=None)

Was können wir beobachten? Die Musik ist eindeutig diatonisch; die zusätzlichen Töne (F#, G# und Bb) sind dagegen äußerst selten. 

Durch den folgenden Befehl wird eine Liste der Tonhöhenhäufigkeiten angezeigt und als csv-Datei exportiert (csv = comma separated variables; lesbar und weiterverarbeitbar u.a. in Excel oder im Text-Editor). Durch den Export lassen sich Tabellen zum Stückvergleich und zur Korpusanalyse generieren. Die csv-Datei wird in dem Ordner gespeichert, den Sie in Ihren Settings als "settings['directoryScratch']" festgelegt haben und kann mit einem Texteditor oder einem Tabellenkalkulationsprogramm (z.B. Excel) geöffnet werden. 

#### 2.2 Tonhöhenklassen

Die Tönhöhen lassen sich in einem zweiten Schritt zu Tonhöhenklassen zusammenfassen. Dadurch wird das Ergebnis übersichtlicher. 

In [ ]:

pitchclass_hist = mp.analyse.pitch_class_histogram(m_df, do_plot=True)

mp.utils.export_as_csv(data=pitchclass_hist, 
                       columns=['Pitch Class','Occurrences'],
                       save_file_name ='pitch_class_hist.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

Durch folgenden Befehlsfolge können die Häufigkeiten der einzelnen Tonhöhenklassen in einer Liste angezeigt und als csv-Datei exportiert werden: 

#### 2.3 Tondauern 

Welche Dauernwerte verwendet Josquin in seiner Missa und wie häufig kommen diese jeweils vor?

In [ ]:
# Für das Dauernwerte-Histogram muss eine andere Befehlsmethode angewendet werden.
# Der graph-Befehl muss mit dem run-Befehl kombiniert werden. 

quarter_dur_hist = mp.analyse.quarterlength_duration_histogram(m_df,
                                                               with_pitch=False, 
                                                               with_pitchclass=False,
                                                               do_plot=True)

mp.utils.export_as_csv(data=quarter_dur_hist, 
                       columns=['Duration', 'Occurrences'],
                       save_file_name ='quarter_duration_hist.csv', 
                       do_save=True,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

Die Notenwerte werden als Vielfache einer Viertel (Quarter Lenght) gezählt. 
Der Notentext besteht somit vorwiegend aus Halben (2.0) und Ganzen (4.0); andere Notenwerte sind seltener. 

#### 2.4 Metrische Akzente: Töne auf unterschiedlichen metrischen Positionen 

Wie deutlich wird das Metrum in den einzelnen Stimmen einer Komposition – durch die Platzierung der Töne auf Taktanfängen und den metrisch wichtigen Positionen innerhalb des Taktes (z.B. der Taktmitte oder auf den Viertelpositionen)? Hierzu lässt sich in eine Liste der Häufigkeiten von Tönen auf den verschiedenen metrischen Positionen in den einzelnen Stimmen anzeigen. 

Die metrischen Akzente folgen für die gängigen Taktarten folgender Gewichtung (jeweils für eine Folge von Achtelnoten): im 4/4-Takt: 1.0, 0.125, 0.25, 0.125, 0.5, 0.125, 0.25, 0.125; im   3/4-Takt: 1.0, 0.25, 0.5, 0.25, 0.5, 0.25 usw.  
Das heißt also: Der Taktanfang erhält mit 1.0 das höchste Gewicht. 0.5 = mittleres Gewicht, 0.25 geringeres Gewicht usw.

In [ ]:
metric_profile_hist = mp.analyse.metric_profile(m_df, with_pitch=False,  do_plot=True)

mp.utils.export_as_csv(data=metric_profile_hist, 
                       columns=['Beat Strength','Occurrences'],
                       save_file_name ='metric_profile_hist.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

In [ ]:
ts_hist = mp.analyse.time_signature_histogram(m_df, do_plot=True,do_adjusted=False)
mp.utils.display_table(data=ts_hist, 
                       columns=['Time Signature', 'Occurrences'])

In [ ]:
ts_hist = mp.analyse.time_signature_histogram(m_df,
                                              do_plot=False, 
                                              do_adjusted=True)
mp.utils.display_table(data=ts_hist, 
                       columns=['Time Signature', 'Occurrences'])

In [ ]:
metric_profile_p_hist = mp.analyse.metric_profile(m_df, with_pitch=True, do_plot=True)

mp.utils.export_as_csv(data=metric_profile_p_hist, 
                       columns=['MIDI', 'Pitch','Beat Strength','Occurrences'],
                       save_file_name ='metric_profile_hist.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

In [ ]:
bs_hist_list = mp.analyse.metric_profile_split_time_signature(m_df, with_pitch=False, do_plot=True)
# TODO: change beat strength to  metric_profile
# TODO: axis start from 1, 2,3,...

In [ ]:

mp.utils.display_table(data=bs_hist_list[0], 
                       columns=['Beat Strength', 'Occurrences'])

In [ ]:

mp.utils.display_table(data=bs_hist_list[1], 
                       columns=['Beat Strength', 'Occurrences'])

In [ ]:
bs_3dhist_list = mp.analyse.metric_profile_split_time_signature(m_df, with_pitch=True, do_plot=True)


In [ ]:
print(f"Number of unique time signature :{len(bs_3dhist_list)}")


In [ ]:
mp.utils.display_table(data=bs_3dhist_list[0], 
                       columns=['MIDI', 'Pitch', 'Beat Strength', 'Occurrences'])

mp.utils.display_table(data=bs_3dhist_list[1], 
                       columns=['MIDI', 'Pitch','Beat Strength', 'Occurrences'])

Im Falle des "3/1-Taktes" entspricht die 1.0 den Taktanfängen, der zweiten und dritten Ganzen-Position im Takt, die 0.25 den verbleibenden Halb-Positionen. 

### 3. Zweidimensionale Häufigkeitsverteilungen

Wir haben uns bereits die Häufigkeiten von Tonhöhen bzw. Tonhöhenklassen angeschaut. Nun könnte man sagen: Längere Töne haben natürlich mehr Gewicht als kurze Töne oder Töne zwischen den Zählzeiten. Diesen Gedanken können wir weiterverfolgen, indem wir uns kombinierte, ‚doppelte‘ oder ‚bivariate‘ Häufigkeitsverteilungen: Also z.B. die Häufigkeiten der Tonhöhen für jeweils die verschiedenen Dauernwerte oder die Häufigkeiten der Tonhöhenklasse jeweils für die verschiedenen metrischen Positionen. Hierum soll es im Folgenden anhand von zwei Beispielen gehen.  
Beispiel 1: Dauern pro Tonhöhenklassen
Gibt es Unterschiede bei den Dauernwerten bzgl. der verschiedenen Tonhöhenklassen?


In [ ]:
dur_pc_hist = mp.analyse.quarterlength_duration_histogram(m_df,
                                                          with_pitch=True,
                                                          do_plot=True)


Was lässt sich aus der Grafik herauslesen?

Beispiel 2: Töne auf den unterschiedlich akzentuierten metrischen Positionen (s. oben, 2.4) im Verhältnis zu Tonhöhenklassen. Hierzu ist wieder ein etwas komplexerer Befehl notwendig: 

### 4. Intervalle im einstimmigen Verlauf 

Gehen wir nun zurück zum Beispiel von Josquin. Wie häufig kommen bestimmte Intervallschritte in den einzelnen Stimmen vor? Sind da alle Stimmen ähnlich gelagert – oder gibt es z.B. im Bass mehr Sprünge, in den Mittelstimmen mehr Schritte? 

Die Beantwortung dieser Frage kann bei music21 nur in mehreren Schritten erfolgen. 

In [ ]:
# Zunächst muss eine Stimme ausgewählt und mit einer neuen Variablen bezeichnet werden. 
# Superius = 0, Altus = 1, Tenor = 2, Bassus = 3
# Wählen wir die Bassstimme. 
v = m_df[['PartID','Part Name']].drop_duplicates().to_numpy()
print(v)

interval_hist = mp.analyse.interval(m_df,
                                    part='2',
                                    do_plot=True)

mp.utils.export_as_csv(data=interval_hist, 
                       columns=['Interval', 'Occurrences'],
                       save_file_name ='interval.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

In [ ]:
# Zunächst muss eine Stimme ausgewählt und mit einer neuen Variablen bezeichnet werden. 
# Superius = 0, Altus = 1, Tenor = 2, Bassus = 3
# Wählen wir die Bassstimme. 


# Adding Filters to the dataframe using python dictionary 

order of the dictionary matters. Every anylyse python methods has the attribute named "filter_dict". 

#### Example :
filter_dict = {'Measure':'1',
          'Part Name':'Superius'}

In [ ]:
filter_dict = {'Measure':'1',
          'Part Name':'Superius'}

filtered_df = mp.analyse.filter(m_df,filter_dict)

filtered_df

Nun der Vergleich mit der Altus-Stimme. Was lässt sich hieraus über die Art der Fortschreitung der beiden Stimmen sagen? Ist sie ähnlich oder unterschiedlich?

In [ ]:
filter_dict_2 = {'Measure':'1-3', 'PartID':'1-2'}
filtered_df_2 = mp.analyse.filter(m_df,filter_dict_2)
filtered_df_2

In [ ]:
filter_dict_2 = { 'PartID':'1-2', 'Measure':'1-3'}
filtered_df_2 = mp.analyse.filter(m_df,filter_dict_2)
filtered_df_2

In [ ]:
filter_dict_2 = {'PartID':'2', 'Measure':'1-3'}
filtered_df_2 = mp.analyse.filter(m_df,filter_dict_2)
filtered_df_2

Mit den folgenden Befehlen lässt sich die Liste der Intervallhäufigkeiten anzeigen exportieren bzw. als Histogramm anzeigen.

In [ ]:
v = m_df[['PartID','Part Name']].drop_duplicates().to_numpy()
print(v)

filter_dict_interval ={'PartID':'1-2', 'Measure':'1-5'}
interval_hist_example = mp.analyse.interval(m_df,
                                    part='all',
                                    do_plot=True, filter_dict = filter_dict_interval)


mp.utils.export_as_csv(data=interval_hist_example, 
                       columns=['Interval', 'Occurrences'],
                       save_file_name ='interval.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

In [ ]:
v = m_df[['PartID','Part Name']].drop_duplicates().to_numpy()
print(v)

filter_dict_interval ={'PartID':'1-2', 'Measure':'1-16'}

interval_hist_example = mp.analyse.interval(m_df,
                                    part='all',filter_dict=filter_dict_interval,
                                    do_plot=True)


mp.utils.export_as_csv(data=interval_hist_example, 
                       columns=['Interval', 'Occurrences'],
                       save_file_name ='interval.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

### 5. Aufgaben

Vergleichende statistische Abfragen über mehrere Stücke:

Wir haben uns die Ergebnisse bislang nur anhand jeweils eines Stückes angeschaut. Wie stellt sich nun aber die Lage dar, wenn wir mehrere Stücke, z.B. mehrere Sätze von Josquin, miteinander – und mit den Stücken eines seiner Zeitgenossen – vergleichen? Gibt es da tatsächlich stilistische Regelmäßigkeiten – oder stilistische Unterschiede?

Wählen Sie Kompositionen Ihrer Wahl (verschiedene Gattungen, Komponisten und Epochen) und vergleichen Sie diese Stücke hinsichtlich der Häufigkeiten von Tonhöhen, Tonhöhenklassen, Notenwerten und Intervallen. Beschreiben Sie die Ergebnisse!